In [1]:
# 패키지 import
import datetime as dt
import pandas as pd
import requests
import time
import urllib.request #
import re 

In [2]:
from bs4 import BeautifulSoup
from pandas import DataFrame
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
from selenium.common.exceptions import NoSuchElementException

In [63]:
from selenium import webdriver
from selenium.webdriver.common.by import By

def naver_webtoon_finish1():
    
    driver = webdriver.Chrome()

    driver.get('http://nid.naver.com/nidlogin.login')
    time.sleep(20)

    url = 'https://comic.naver.com/webtoon?tab=finish'

    results = []
    link_list = []
    day_list = []  # 요일
    rank_list = []  # 순위
    title_list = []  # 제목
    rating_list = []  # 별점

    story_list = []  # 스토리
    keywords_list = []  # 키워드
    like_list = []  # 관심
    num_list = []  # 몇화까지
    age_list = []  # 이용가
    
    
    for i in range(1, 51):

        driver.get(url)
        time.sleep(2)
        driver.implicitly_wait(5)
        
        wait = WebDriverWait(driver, 10)
        button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#content > div:nth-child(1) > div > div.ComponentHead__tab_control--R1TyL > button:nth-child(1)')))
        button.click()
        time.sleep(2)
        print("버튼이 클릭되었습니다.")


        # 각 웹툰의 링크 가져오기
        link_selector = f'#content > div:nth-child(1) > ul > li:nth-child({i}) > a'

        link_element = driver.find_element(By.CSS_SELECTOR, link_selector)
        webtoon_link = link_element.get_attribute('href')
        link_list.append(webtoon_link)


        # 웹툰이름, 작가, 별점을 가져올 수 있음. -> information에 들어있다.
        selector = f'#content > div:nth-child(1) > ul > li:nth-child({i}) > div'

        element = driver.find_element(By.CSS_SELECTOR, selector)
        information = element.text.strip().split('\n')
        if len(information) == 5:
            title = information[1]
        else:
            title = information[0]
        rating = information[-1]

        day_list.append('finish')
        title_list.append(title)
        rating_list.append(rating)
        rank_list.append(i)

        print(information)

     
    for j in range(1,51):# 상세 페이지로 이동
        driver.get(link_list[j])
        driver.implicitly_wait(10)
        time.sleep(2)

        html = driver.page_source
        soup = bs(html, 'html.parser')

        ### 장르, 요약, 키워드, 관심, 몇화
        xpath_episode_info = '//*[@id="content"]/div[1]/div/div[2]/p'
        xpath_keywords = '//*[@id="content"]/div[1]/div/div[2]/div/div'
        xpath_interest = '//*[@id="content"]/div[2]/div/button[1]/span[2]'
        xpath_age = '//*[@id="content"]/div[1]/div/div[1]/em' 
        xpath_total_episodes = '//div[@class="EpisodeListView__count--fTMc5"]'
        
        try:
            episode_info = driver.find_element(By.XPATH, xpath_episode_info).text
        except NoSuchElementException:
            episode_info = 'N/A'

        try:
            keywords = driver.find_element(By.XPATH, xpath_keywords).text
        except NoSuchElementException:
            keywords = 'N/A'

        try:
            interest = driver.find_element(By.XPATH, xpath_interest).text
        except NoSuchElementException:
            interest = 'N/A'

        try:
            age_text = driver.find_element(By.XPATH, xpath_age).text
            age = age_text.split('\n')[-1].strip()  # 마지막 줄만 추출하여 공백 제거
        except NoSuchElementException:
            age = 'N/A'

        print(interest) 

        try:
            total_episodes_text = driver.find_element(By.XPATH, xpath_total_episodes).text
            # 숫자 부분만 추출
            total_episodes = ''.join(filter(str.isdigit, total_episodes_text))
        except NoSuchElementException:
            total_episodes = 'N/A'

        story_list.append(episode_info)  # 스토리
        keywords_list.append(keywords)  # 키워드
        like_list.append(interest)  # 관심
        num_list.append(total_episodes)  # 몇화까지
        age_list.append(age)  # 이용가

        time.sleep(1)
        driver.implicitly_wait(2)


    driver.quit()
    
    data = {
        'Title': title_list,
        'Link': link_list,
        'Day': day_list,
        'Rank': rank_list,
        'Rating': rating_list,
        'Story': story_list,
        'Keywords': keywords_list,
        'Likes': like_list,
        'Num': num_list,
        'Age': age_list
    }
    return pd.DataFrame(data)

In [65]:
from selenium import webdriver
from selenium.webdriver.common.by import By

def naver_webtoon_finish_link():
    
    driver = webdriver.Chrome()

    driver.get('http://nid.naver.com/nidlogin.login')
    time.sleep(15)

    url = 'https://comic.naver.com/webtoon?tab=finish'

    results = []
    link_list = []
    day_list = []  # 요일
    rank_list = []  # 순위
    title_list = []  # 제목
    rating_list = []  # 별점
    
    for j in [1,3,4]:
        
        driver.get(url)
        time.sleep(2)
        driver.implicitly_wait(5)

        wait = WebDriverWait(driver, 10)
        button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#content > div:nth-child(1) > div > div.ComponentHead__tab_control--R1TyL > button:nth-child(1)')))
        button.click()
        time.sleep(2)
        print("버튼이 클릭되었습니다.")
        
        for i in range(1, 51):
            
            # 각 웹툰의 링크 가져오기
            link_selector = f'#content > div:nth-child(1) > ul > li:nth-child({i}) > a'

            link_element = driver.find_element(By.CSS_SELECTOR, link_selector)
            webtoon_link = link_element.get_attribute('href')
            link_list.append(webtoon_link)

            # 웹툰이름, 작가, 별점을 가져올 수 있음. -> information에 들어있다.
            selector = f'#content > div:nth-child(1) > ul > li:nth-child({i}) > div'

            element = driver.find_element(By.CSS_SELECTOR, selector)
            information = element.text.strip().split('\n')
            if len(information) == 5:
                title = information[1]
            else:
                title = information[0]
            rating = information[-1]

            day_list.append('finish')
            title_list.append(title)
            rating_list.append(rating)
            rank_list.append(i)

            print(information)

    driver.quit()
    
    data = {
        'Title': title_list,
        'Link': link_list,
        'Day': day_list,
        'Rank': rank_list,
        'Rating': rating_list,
    }
    return pd.DataFrame(data)

In [97]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

def naver_webtoon_finish_link():
    driver = webdriver.Chrome()
    url = 'https://comic.naver.com/webtoon?tab=finish'

    results = []
    link_list = []
    day_list = []  # 요일
    rank_list = []  # 순위
    title_list = []  # 제목
    rating_list = []  # 별점

    driver.get(url)
    time.sleep(2)
    driver.implicitly_wait(5)

    # '종료' 탭 버튼 클릭
    wait = WebDriverWait(driver, 10)
    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#content > div:nth-child(1) > div > div.ComponentHead__tab_control--R1TyL > button:nth-child(1)')))
    button.click()
    time.sleep(2)
    print("버튼이 클릭되었습니다.")

    for _ in range(50):
    # 페이지의 끝까지 스크롤
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        # 페이지가 로드될 시간을 줍니다
        time.sleep(1)

    # 웹툰 정보 수집
    for i in range(1, 201):
        try:
            # 각 웹툰의 링크 가져오기
            link_selector = f'#content > div:nth-child(1) > ul > li:nth-child({i}) > a'
            link_element = driver.find_element(By.CSS_SELECTOR, link_selector)
            webtoon_link = link_element.get_attribute('href')
            link_list.append(webtoon_link)

            # 웹툰 이름, 작가, 별점을 가져오기
            selector = f'#content > div:nth-child(1) > ul > li:nth-child({i}) > div'
            element = driver.find_element(By.CSS_SELECTOR, selector)
            information = element.text.strip().split('\n')
            if len(information) == 5:
                title = information[1]
            else:
                title = information[0]
            rating = information[-1]

            day_list.append('finish')
            title_list.append(title)
            rating_list.append(rating)
            rank_list.append(i)

            print(information)
        except NoSuchElementException:
            # 더 이상 항목이 없으면 루프 종료
            print(f'Element for index {i} not found.')
            break

    driver.quit()

    data = {
        'Title': title_list,
        'Link': link_list,
        'Day': day_list,
        'Rank': rank_list,
        'Rating': rating_list,
    }
    return pd.DataFrame(data)


In [98]:
df_link = naver_webtoon_finish_link()

버튼이 클릭되었습니다.
['휴재', '광마회귀', 'JP / 이히 / 유진성', '별점', '9.97']
['2024 내일 뭐 입지?', '네이버웹툰 작가', '별점', '9.91']
['휴재', '격기3반', '이학', '별점', '9.32']
['삼국지톡', '무적핑크 / 이리', '별점', '9.80']
['올가미', '해무리', '별점', '9.97']
['2024 너네 뭐 먹니?', '네이버웹툰 작가', '별점', '9.82']
['휴재', '마도귀환록', '팀 더 제이 / MISANG / 한유림', '별점', '9.92']
['휴재', '무직백수 계백순', '지발', '별점', '9.90']
['어린 상사', 'LICO / 박수정', '별점', '9.89']
['휴재', '호랑이형님', '이상규', '별점', '9.91']
['휴재', '무림서부', '임팩 / 정한길 / 컵라면.', '별점', '9.95']
['로잘린 보가트', '삼월에 / 마하 / 하노HANO', '별점', '9.97']
['캐슬', '정연', '별점', '9.97']
['선천적 얼간이들', '가스파드', '별점', '9.96']
['우리 집 고양이 보고 갈래?', '삼태', '별점', '9.49']
['아슈타르테', '미세초 / SOON / 안다온', '별점', '9.87']
['더 게이머', '성상영 / 상아', '별점', '8.40']
['예명여고', '육공', '별점', '9.94']
['룸9', '은작', '별점', '9.73']
['해골협객', '성불예정 / 준1', '별점', '9.42']
['대신 살쪄주는 여자', '롸나', '별점', '7.82']
['어느 백작 영애의 이중생활', 'Jeu / 킴뽀 / 최아리', '별점', '9.96']
['이말년씨리즈', '이말년', '별점', '8.96']
['휴재', '투신전생기', '청담', '별점', '9.95']
['휴재', '저 그런 인재 아닙니다', '에몽 / 머나 / 덴피', '별점', '9.97']
['휴재', '

In [99]:
df_link

,Title,Link,Day,Rank,Rating
0,광마회귀,https://comic.naver.com/webtoon/list?titleId=7...,finish,1,9.97
1,2024 내일 뭐 입지?,https://comic.naver.com/webtoon/list?titleId=8...,finish,2,9.91
2,격기3반,https://comic.naver.com/webtoon/list?titleId=7...,finish,3,9.32
3,삼국지톡,https://comic.naver.com/webtoon/list?titleId=7...,finish,4,9.80
4,올가미,https://comic.naver.com/webtoon/list?titleId=7...,finish,5,9.97
...,...,...,...,...,...
195,와탕카,https://comic.naver.com/webtoon/list?titleId=1...,finish,196,9.64
196,급식아빠,https://comic.naver.com/webtoon/list?titleId=7...,finish,197,9.75
197,DEY 호러채널,https://comic.naver.com/webtoon/list?titleId=6...,finish,198,9.95
198,천치전능,https://comic.naver.com/webtoon/list?titleId=7...,finish,199,9.93


In [105]:
def naver_webtoon_finish_information(df):
    story_list = []  # 스토리
    keywords_list = []  # 키워드
    like_list = []  # 관심
    num_list = []  # 몇화까지
    age_list = []  # 이용가
    
    driver = webdriver.Chrome()
    
    # 미리 로그인 (성인웹툰 접근용 )
    driver.get('http://nid.naver.com/nidlogin.login')
    time.sleep(20)
    
    for i in range(1,201):
        driver.get(df["Link"][i-1])
        driver.implicitly_wait(10)
        time.sleep(2)

        html = driver.page_source
        soup = bs(html, 'html.parser')

        ### 장르, 요약, 키워드, 관심, 몇화
        xpath_episode_info = '//*[@id="content"]/div[1]/div/div[2]/p'
        xpath_keywords = '//*[@id="content"]/div[1]/div/div[2]/div/div'
        xpath_interest = '//*[@id="content"]/div[2]/div/button[1]/span[2]'
        xpath_age = '//*[@id="content"]/div[1]/div/div[1]/em' 
        xpath_total_episodes = '//div[@class="EpisodeListView__count--fTMc5"]'

        try:
            episode_info = driver.find_element(By.XPATH, xpath_episode_info).text
        except NoSuchElementException:
            episode_info = 'N/A'

        try:
            keywords = driver.find_element(By.XPATH, xpath_keywords).text
        except NoSuchElementException:
            keywords = 'N/A'

        try:
            interest = driver.find_element(By.XPATH, xpath_interest).text
        except NoSuchElementException:
            interest = 'N/A'

        try:
            age_text = driver.find_element(By.XPATH, xpath_age).text
            age = age_text.split('\n')[-1].strip()  # 마지막 줄만 추출하여 공백 제거
        except NoSuchElementException:
            age = 'N/A'

        print(interest) 

        try:
            total_episodes_text = driver.find_element(By.XPATH, xpath_total_episodes).text
            # 숫자 부분만 추출
            total_episodes = ''.join(filter(str.isdigit, total_episodes_text))
        except NoSuchElementException:
            total_episodes = 'N/A'

        story_list.append(episode_info)  # 스토리
        keywords_list.append(keywords)  # 키워드
        like_list.append(interest)  # 관심
        num_list.append(total_episodes)  # 몇화까지
        age_list.append(age)  # 이용가

    driver.quit()
    
    data = {
        'Story': story_list,
        'Keywords': keywords_list,
        'Likes': like_list,
        'Num': num_list,
        'Age': age_list
    }
    data = pd.DataFrame(data)
    data2 = pd.concat([df,data],axis=1)
    return data2

In [106]:
df_final = naver_webtoon_finish_information(df_link)

468,797
66,978
314,841
161,545
535,758
29,398
82,529
200,544
92,892
870,316
74,279
152,698
449,442
447,693
169,692
138,123
384,521
87,893
36,594
28,711
38,884
59,814
158,254
464,581
269,732
952,165
336,297
239,822
417,314
66,473
4,016
448,359
95,178
203,810
259,708
72,225
66,197
71,122
74,149
56,788
409,806
218,847
207,298
37,686
58,953
33,234
322,794
255,925
86,109
2,582,188
858,000
10,440
284,521
514,705
150,810
445,268
9,821
144,038
92,083
398,410
191,004
325,889
297,041
644,320
86,288
47,041
48,126
386,594
203,621
84,380
968,836
86,902
119,739
78,577
503,300
43,431
387,566
1,062,581
161,840
94,177
95,254
247,991
437,757
750,646
45,484
561,628
84,311
112,006
435,962
91,931
46,495
108,045
170,467
168,469
761,784
59,607
13,451
31,998
405,061
38,408
721,718
273,481
395,493
96,844
406,826
193,559
53,618
57,897
16,629
36,565
189,962
171,813
1,754,443
106,020
125,015
97,102
647,578
403,451
80,264
1,835
44,669
260,031
728,959
278,503
413,799
265,356
143,380
113,751
783,832
57,321
43,083
13

In [107]:
df_final

,Title,Link,Day,Rank,Rating,Story,Keywords,Likes,Num,Age
0,광마회귀,https://comic.naver.com/webtoon/list?titleId=7...,finish,1,9.97,무공에 미친 광마 이자하.\n그는 마교 교주의 천옥을 훔쳐 쫓기던 중 벼랑에서 떨어...,#무협/사극\n#사이다\n#세계관\n#회귀\n#판무\n#소설원작,"468,797",121,15세 이용가
1,2024 내일 뭐 입지?,https://comic.naver.com/webtoon/list?titleId=8...,finish,2,9.91,'옷은 많은데.. 왜 맨날 입을 옷이 없지..???' 최애캐가 골라주는 내일의 코디!,#일상\n#완결무료\n#완결일상,"66,978",32,전체연령가
2,격기3반,https://comic.naver.com/webtoon/list?titleId=7...,finish,3,9.32,한국이 세계 격투기의 중심이 된 세상. 전국 탑 클래스 격투기 학생들만 모인 남일고...,#스포츠\n#환골탈태\n#격투기,"314,841",259,15세 이용가
3,삼국지톡,https://comic.naver.com/webtoon/list?titleId=7...,finish,4,9.80,"""유비, 조조, 손권 등 영웅들이 활약하던 후한, 그 난세에 핸드폰이 있었다면?\n...",#무협/사극\n#유료화임박\n#완결무료\n#완결무협/사극,"161,545",597,12세 이용가
4,올가미,https://comic.naver.com/webtoon/list?titleId=7...,finish,5,9.97,"""일주일에 단 한 번, 내가 지시하는 곳으로 데려오기만 하면 돼요.""\n정체불명의 ...",#드라마\n#유료화임박\n#폭스남\n#집착물\n#고자극로맨스\n#혐관로맨스\n#인외존재,"535,758",204,15세 이용가
...,...,...,...,...,...,...,...,...,...,...
195,와탕카,https://comic.naver.com/webtoon/list?titleId=1...,finish,196,9.64,"와탕카='만세','지화자'\n\n좋은 패가 들어오면 외치는 단어 와탕카! 우주인 작...",#개그\n#완결무료\n#완결개그,"4,835",706,12세 이용가
196,급식아빠,https://comic.naver.com/webtoon/list?titleId=7...,finish,197,9.75,<부활남> 김재한 작가의 신작! 딸바보 웹툰작가의 갑작스러운 죽음.\n학원액션 만화...,#액션\n#사이다\n#하이틴\n#학원물,"347,175",127,15세 이용가
197,DEY 호러채널,https://comic.naver.com/webtoon/list?titleId=6...,finish,198,9.95,[데드데이즈] DEY작가의 새로운 공포 단편선\n기발하고 소름 돋는 무서운 이야기들...,#스릴러\n#성인웹툰\n#완결무료\n#완결스릴러,"78,606",29,18세 이용가
198,천치전능,https://comic.naver.com/webtoon/list?titleId=7...,finish,199,9.93,학창 시절 매일 폭력 사건을 일으키던 문제아 김전지는 오토바이 사고로 다리를 다쳐 ...,#드라마\n#완결드라마,"64,293",121,12세 이용가


In [108]:
df_final.to_excel('naverwebtoon_final.xlsx')